In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [5]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1,
              0, 1, 0, 1, 1], dtype = np.float64)

In [7]:
def calc_std_fit(x):
    res = (x - x.mean()) / x.std()
    return res

In [9]:
X_st = X.copy()
X_st[2, :] = calc_std_fit(X[1, :])

In [12]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [19]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [13]:
def eval_model_(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    return W, err, alpha, iterations

In [15]:
best_params = None
err_min = 1000
i = 0
for iterations in [4000, 5000, 3500, 2000, 3000]:
    for alpha in [1e-3, 1e-8, 1e-2, 0.005, 1e-5, 1e-4]:
        params = W, err, alpha, iterations = eval_model_(X_st, y, iterations=iterations, alpha=alpha)
        if err < err_min:
            err_min = err
            best_params = params

print(f'Итерации {params[-1]}, alpha {params[-2]}')

Итерации 3000, alpha 0.0001


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [21]:
def calc_pred_proba(W, X):
    y_pred = np.dot(W, X)
    y_pred_proba = sigmoid(y_pred)
    return y_pred_proba
weights = 0

In [23]:
W = eval_model(X_st, y, iterations=3000, alpha=1e-4)

300 [ 0.48496327 -0.17384257  0.64589355  1.50575204] 1.1931571091358864
600 [ 0.47340821 -0.20906258  0.64404467  1.48875145] 1.1376059352778145
900 [ 0.46206317 -0.24385659  0.64215292  1.47205973] 1.0835500530355255
1200 [ 0.45094443 -0.27813585  0.64023486  1.45571533] 1.031226861538826
1500 [ 0.44007076 -0.31178459  0.63831445  1.43976514] 0.9809336291045673
1800 [ 0.42946372 -0.34465401  0.63642475  1.42426596] 0.9330340710859316
2100 [ 0.41914781 -0.37655851  0.63460935  1.40928523] 0.8879543554013178
2400 [ 0.40914981 -0.40727726  0.63292223  1.39489992] 0.8461598024239766
2700 [ 0.39949726 -0.43656504  0.63142538  1.38119251] 0.808105289969183
3000 [ 0.39021583 -0.46417456  0.63018332  1.36824355] 0.7741616988816424


4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [31]:
def calc_pred(w, x, threshold=0.5):
    probs = calc_pred_proba(w, x)
    return [1.0 if z >= threshold else 0 for z in sigmoid(probs)]

In [32]:
y_pred = calc_pred(W, X_st)
y_pred

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [33]:
accuracy = 1/len(y) * np.sum([1 if y[i] == y_pred[i] else 0 for i in range(len(y))])
accuracy

0.5

In [34]:
def confusion_matrix(y, y_pred):
    b = list(reversed(np.unique(y)))
    m = [[0 for x in range(len(b))] for x in range(len(b))]
    
    for i, yi in enumerate(y):
        m[b.index(yi)][b.index(y_pred[i])] += 1
        
    return m

In [35]:
cm = confusion_matrix(y, y_pred)
cm

[[5, 0], [5, 0]]

In [39]:
precision = cm[0][0] / (cm[0][0] + cm[0][1])
precision

1.0

In [40]:
recall = cm[0][0] / (cm[0][0] + cm[1][0])
recall

0.5

In [41]:
f1_score = (2 * precision * recall) / (precision + recall)
f1_score

0.6666666666666666

6. Могла ли модель переобучиться? Почему?

Да, могла. Похоже переобучение